In [1]:
import torch
from lib.data import Constants

In [2]:
def get_data_trees(trees):
    data_trees = []
    for t_json in trees:
        for k, node in t_json.items():
            # if 'parent' not in node:
            #     root_idx ='#'
            #     break
            if node['parent'] == None and node['node'] != 'StatementExpression':
                root_idx = k
        # if root_idx == "#":
        #     continue
        tree = json2tree_binary(t_json, Tree(), root_idx)
        data_trees.append(tree)

    return data_trees 

def json2tree_binary(tree_json, tree, idx, prev=None):
    if prev == None:
        tree.parent = prev
        tree.idx = idx
    children = [c for c in tree_json[idx]['children'] if c.startswith(Constants.NODE_FIX)]
    if len(children) == 2:
        for c in children:
            t = Tree()
            t.parent = prev
            t.idx = c
            t.num_children = 0
            k = json2tree_binary(tree_json, t, c, tree)
            tree.add_child(k)
    else:
        tree.parent = prev
        tree.num_children = 0
        # print "tree_json[idx]['children']: "
        # print tree_json[idx]['children']
        tree.content = tree_json[idx]['children'][0]
    return tree

In [3]:
class Tree(object):
    def __init__(self):
        self.parent = None
        self.num_children = 0
        self.children = list()
        self.leaf_states = list()
        self._leaf_count = 0
        self._leaf_contents = []
        self.content = None

    def add_child(self,child):
        child.parent = self
        self.num_children += 1
        self.children.append(child)

    def size(self):
        # if getattr(self,'_size'):
        #     return self._size
        count = 1
        for i in range(self.num_children):
            count += self.children[i].size()
        self._size = count
        return self._size

    def depth(self):
        # if getattr(self,'_depth'):
        #     return self._depth
        count = 0
        if self.num_children>0:
            for i in range(self.num_children):
                child_depth = self.children[i].depth()
                if child_depth>count:
                    count = child_depth
            count += 1
        self._depth = count
        return self._depth

    def leaf_count_(self):
        # count = 0
        if not self.children:
            self._leaf_count = 1
        else:
            for c in self.children:
                leaf_count = c.leaf_count()
                self._leaf_count += leaf_count
                # print "self._leaf_count: ", self._leaf_count
        # self._leaf_count = count
        return self._leaf_count

    def leaf_count(self):
        count = 0
        # if not self.children:
        #     self._leaf_count = 1
        # else:
        # if not self.children:
        #     count += 1
        if self.children:
            for c in self.children:
                leaf_count = c.leaf_count()
                count += leaf_count
                # if leaf_count>count:
                #     count = leaf_count

        else:
            count += 1
        # self._leaf_count = count
        return count
                # self._leaf_count += leaf_count
                # print "self._leaf_count: ", self._leaf_count
        # self._leaf_count = count
        # return self._leaf_count

    def leaf_contents(self):
        if not self.children:
            # print "self.content: "
            # print self.content
            self._leaf_contents.append(self.content)
        else:
            for c in self.children:
                leaf_content = c.leaf_contents()
                self._leaf_contents.extend(leaf_content)

        return self._leaf_contents

In [7]:
data_path = '/mnt/yuxuan/Code_summarization/csn_dataset/train_data/processed_filter.train.pt2'

In [8]:
data = torch.load(data_path)

In [9]:
data.keys()

dict_keys(['train_xe', 'test', 'train_pg', 'valid', 'dicts'])

In [10]:
binary_tree = get_data_trees(data['train_xe']['trees'])

In [11]:
len(binary_tree)

430581

In [12]:
print(binary_tree[0].depth())
print(binary_tree[0].leaf_count())

14
20


In [13]:
depth = []
length = []
for t in binary_tree:
    length.append(t.leaf_count())
    depth.append(t.depth())
    

In [14]:
sum(depth) / 430580

10.581831483115797

In [15]:
sum(length) / 430580

23.223993218449532

In [16]:
from collections import Counter

In [17]:
l_count = Counter(length)
d_count = Counter(depth)

In [18]:
l_sort = sorted(l_count.items(), key=lambda x:x[0])

In [19]:
def get_percent_data(data, percent, total):
    filter_num = percent * total
    s = 0
    for l in data:
        if s+l[1] > filter_num:
            return l[0]
        else:
            s += l[1]
    return l[0]

In [28]:
total = 430580
eighty_percent_num = get_percent_data(l_sort, 0.8, total)
ninty_percent_num = get_percent_data(l_sort, 0.9, total)
nintyfive_percent_num = get_percent_data(l_sort, 0.95, total)
percent99_num = get_percent_data(l_sort, 0.999, total)

In [29]:
print(eighty_percent_num)
print(ninty_percent_num)
print(nintyfive_percent_num)
print(percent99_num)

30
47
70
393


In [48]:
l_sort

[(2, 987),
 (3, 2164),
 (4, 5400),
 (5, 16574),
 (6, 28134),
 (7, 26248),
 (8, 24673),
 (9, 29209),
 (10, 23872),
 (11, 19995),
 (12, 20385),
 (13, 18180),
 (14, 13836),
 (15, 14825),
 (16, 11497),
 (17, 11512),
 (18, 9337),
 (19, 9477),
 (20, 7867),
 (21, 7787),
 (22, 6587),
 (23, 6551),
 (24, 5552),
 (25, 5490),
 (26, 4815),
 (27, 4724),
 (28, 4633),
 (29, 4036),
 (30, 3909),
 (31, 3634),
 (32, 3473),
 (33, 3106),
 (34, 2877),
 (35, 2861),
 (36, 2614),
 (37, 2541),
 (38, 2323),
 (39, 2273),
 (40, 1955),
 (41, 2033),
 (42, 1931),
 (43, 1938),
 (44, 1769),
 (45, 1626),
 (46, 1507),
 (47, 1575),
 (48, 1433),
 (49, 1402),
 (50, 1274),
 (51, 1228),
 (52, 1152),
 (53, 1142),
 (54, 1127),
 (55, 994),
 (56, 964),
 (57, 1002),
 (58, 926),
 (59, 830),
 (60, 874),
 (61, 784),
 (62, 799),
 (63, 697),
 (64, 698),
 (65, 712),
 (66, 699),
 (67, 643),
 (68, 605),
 (69, 589),
 (70, 545),
 (71, 546),
 (72, 519),
 (73, 485),
 (74, 477),
 (75, 483),
 (76, 436),
 (77, 461),
 (78, 420),
 (79, 418),
 (80, 

### To explore calculate different leaf_count influences cost time 

**TEST 1**

90 datas where leaf_count < 30 and 10 datas where leaf_count > 2000<br>
100 datas where leaf_count < 30

In [62]:
def bulid_test_index(num1, num2, binary_tree):
    count = 0
    test1 = []
    test2 = []
    for i, t in enumerate(binary_tree):
        if count >= 110:
            break
        if t.leaf_count() < num1 and len(test1_2) < 100:
            test1_2.append(i)
            count += 1
        elif t.leaf_count() > num2 and len(test1_1) < 10:
            test1_1.append(i)
            count += 1
    test_data1_index = test1_2[:90] + test1_1
    test_data2_index = test1_2
    return test_data1_index, test_data2_index

In [51]:
count = 0
test1_1 = []
test1_2 = []
for i, t in enumerate(binary_tree):
    if count >= 110:
        break
    if t.leaf_count() < 30 and len(test1_2) < 100:
        test1_2.append(i)
        count += 1
    elif t.leaf_count() > 2000 and len(test1_1) < 10:
        test1_1.append(i)
        count += 1


In [57]:
test1_data1_index = test1_2[:90] + test1_1
test1_data2_index = test1_2

In [58]:
def build_test_data(data_index, data):
    test_data = {'train_xe': {}}
    src = []
    tgt = []
    trees = []
    for idx in data_index:
        src.append(data['train_xe']['src'][idx])
        tgt.append(data['train_xe']['tgt'][idx])
        trees.append(data['train_xe']['trees'][idx])
    test_data['train_xe']['src'] = src
    test_data['train_xe']['tgt'] = tgt
    test_data['train_xe']['trees'] = trees
    test_data['test'] = test_data['train_xe']
    test_data['train_pg'] = test_data['train_xe']
    test_data['valid'] = test_data['train_xe']
    return test_data

In [60]:
test1_data1 = build_test_data(test1_data1_index, data)
test2_data2 = build_test_data(test1_data2_index, data)


**TEST 2**<br>

90 datas where leaf_count < 30 and 10 datas where 30 < leaf_count < 47<br>
100 datas where leaf_count < 30

**TEST 3**<br>

90 datas where leaf_count < 47 and 10 datas where 47 < leaf_count < 70<br>
100 datas where leaf_count < 47